This notebook is for performing transfer learning

In [1]:
import input_util as ip
import transfer_models as mdl
import torch
import torch.optim as optim
import torch.nn as nn

# preliminaries:
input_size = 224 # for RESNET
model_name = 'resnet'
num_output = 6 
feature_extract = True

# Load the Data

In [2]:

train_loader,val_loader = ip.load_dataset(input_size)

dataloaders_dict = {}
dataloaders_dict['train']= train_loader
dataloaders_dict['val'] = val_loader



num image files: 61614
shape of label data: 61614x6


# Initialize the Model

In [7]:
model_ft, input_size = mdl.initialize_model(model_name, num_output, feature_extract, use_pretrained=False)
print(model_ft)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('using device:')
print(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=F

# Define Optimizer

In [8]:
# Send the model to GPU
model_ft = model_ft.to(device)

# Gather the parameters to be optimized/updated in this run. If we are
#  finetuning we will be updating all parameters. However, if we are
#  doing feature extract method, we will only update the parameters
#  that we have just initialized, i.e. the parameters with requires_grad
#  is True.

params_to_update = model_ft.parameters()
print("Params to learn:")
if feature_extract:
    params_to_update = []
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            print("\t",name)

# Observe that all parameters are being optimized
#optimizer_ft = optim.SGD(params_to_update, lr=0.001, momentum=0.9)
optimizer_ft = optim.Adam(params_to_update, lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)

# Setup the loss fxn
criterion = nn.MSELoss(reduction='mean')


Params to learn:
	 fc.weight
	 fc.bias


# Training Phase

In [9]:
num_epochs = 5

# Train and evaluate
model_ft, hist = mdl.train_model(model_ft, dataloaders_dict, device, criterion, optimizer_ft, num_epochs=num_epochs,print_every = 5 , is_inception=(model_name=="inception"))

Epoch 0/4
----------
loss for iteration 5: 1.747764
loss for iteration 10: 0.947446
loss for iteration 15: 0.456539
loss for iteration 20: 0.210907
loss for iteration 25: 0.081208
loss for iteration 30: 0.031615
loss for iteration 35: 0.015048
loss for iteration 40: 0.012143
loss for iteration 45: 0.013414
loss for iteration 50: 0.010825
loss for iteration 55: 0.010088
loss for iteration 60: 0.009258
loss for iteration 65: 0.009586
loss for iteration 70: 0.008290
loss for iteration 75: 0.010143
loss for iteration 80: 0.008158
train Loss: 0.0054
loss for iteration 5: 0.005929
loss for iteration 10: 0.006089
loss for iteration 15: 0.007450
loss for iteration 20: 0.007680
loss for iteration 25: 0.007383
loss for iteration 30: 0.006722
loss for iteration 35: 0.007208
loss for iteration 40: 0.008568
loss for iteration 45: 0.008988
loss for iteration 50: 0.006768
loss for iteration 55: 0.006560
loss for iteration 60: 0.005638
loss for iteration 65: 0.007734
loss for iteration 70: 0.007019
lo